<a href="https://colab.research.google.com/github/nickchak21/Images_Model_Gate_Detection/blob/master/Autonomous_submarine_gate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autonomous Submarine Gate Detection Notebook

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [0]:
!pip install -U --pre tensorflow=="1.*"

Make sure you have `pycocotools` installed

In [0]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [0]:
!git clone https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10.git

In [0]:
mv  -v TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/* models/research/object_detection

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

In [0]:
!pip install pillow
!pip install lxml
!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib
!pip install pandas
!pip install opencv-python

In [0]:
!pip install protobuf

Loading training and testing images and labels, getting rid of old train/test images and labels

In [0]:
!git clone https://github.com/nickchak21/Images_Model_Gate_Detection.git

In [0]:
!unzip -uq "Images_Model_Gate_Detection/approach1_frames_train.zip" -d "models/research/object_detection/images/train"

In [0]:
!unzip -uq "Images_Model_Gate_Detection/approach1_frames_test.zip" -d "models/research/object_detection/images/test"

In [0]:
!ls models/research/object_detection/images/train

In [0]:
!rm models/research/object_detection/images/train/*.JPG

In [0]:
!rm models/research/object_detection/images/train/*.xml

In [0]:
!rm models/research/object_detection/images/train/*.jpg

In [0]:
!mv models/research/object_detection/images/train/approach1_frames_train/* models/research/object_detection/images/train

In [0]:
!rmdir models/research/object_detection/images/train/approach1_frames_train

In [0]:
!rm models/research/object_detection/images/test/*
#!rm models/research/object_detection/images/test/*

In [0]:
#!rm models/research/object_detection/images/test/*.jpg

In [0]:
!mv models/research/object_detection/images/test/approach1_frames_test/* models/research/object_detection/images/test
!rmdir models/research/object_detection/images/test/approach1_frames_test

In [0]:
!ls models/research/object_detection/images/test

In [0]:
!rm models/research/object_detection/images/train/frame341.jpg models/research/object_detection/images/train/frame354.jpg models/research/object_detection/images/train/frame335.jpg models/research/object_detection/images/train/frame336.jpg models/research/object_detection/images/train/frame352.jpg models/research/object_detection/images/train/frame347.jpg models/research/object_detection/images/train/frame337.jpg models/research/object_detection/images/train/frame343.jpg models/research/object_detection/images/train/frame338.jpg models/research/object_detection/images/train/frame334.jpg models/research/object_detection/images/train/frame339.jpg models/research/object_detection/images/train/frame356.jpg models/research/object_detection/images/train/frame340.jpg models/research/object_detection/images/train/frame350.jpg models/research/object_detection/images/train/frame344.jpg models/research/object_detection/images/train/frame346.jpg models/research/object_detection/images/train/frame342.jpg models/research/object_detection/images/train/frame355.jpg models/research/object_detection/images/train/frame349.jpg models/research/object_detection/images/train/frame345.jpg

In [0]:
!rm models/research/object_detection/images/test/frame353.jpg models/research/object_detection/images/test/frame351.jpg models/research/object_detection/images/test/frame348.jpg

In [0]:
!unzip -uq "Images_Model_Gate_Detection/approach1_labels_train.zip" -d "models/research/object_detection/images/train"

In [0]:
!unzip -uq "Images_Model_Gate_Detection/approach1_labels_test.zip" -d "models/research/object_detection/images/test"

In [0]:
!mv models/research/object_detection/images/train/approach1_labels_train/* models/research/object_detection/images/train
!mv models/research/object_detection/images/test/approach1_labels_test/* models/research/object_detection/images/test

!rmdir models/research/object_detection/images/train/approach1_labels_train
!rmdir models/research/object_detection/images/test/approach1_labels_test

In [0]:
!ls models/research/object_detection/images/train

In [0]:
!ls models/research/object_detection/images/test

Loading pre-trained model in correct directory

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!tar -C models/research/object_detection -zxvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

Deleting other extra files in object_detection directory

In [0]:
!rm models/research/object_detection/training/*
!rm models/research/object_detection/inference_graph/*

Generate csv file for image labels

In [0]:
%pycat models/research/object_detection/xml_to_csv.py

In [0]:
%%writefile models/research/object_detection/xml_to_csv.py
frames_less_200 = []
frames_greater_200 = []

for i in range(0,200):
    frames_less_200.append('frame{}.jpg'.format(i))

for i in range(200,365):
    frames_greater_200.append('frame{}.jpg'.format(i))

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        # print(root.find('filename').text)
        for member in root.findall('object'):
            if root.find('filename').text in frames_less_200:
                value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        member[0].text,
                        int(member[5][0].text),
                        int(member[5][1].text),
                        int(member[5][2].text),
                        int(member[5][3].text)
                        )
            else:
                value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        member[0].text,
                        int(member[4][0].text),
                        int(member[4][1].text),
                        int(member[4][2].text),
                        int(member[4][3].text)
                        )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    for folder in ['train','test']:
        image_path = os.path.join('models/research/object_detection', ('images/' + folder))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv(('models/research/object_detection/images/' + folder + '_labels.csv'), index=None)
        print('Successfully converted xml to csv.')


main()

In [0]:
!python models/research/object_detection/xml_to_csv.py

In [0]:
!cat models/research/object_detection/images/test_labels.csv

Generating TF Records

In [0]:
%pycat models/research/object_detection/generate_tfrecord.py

In [0]:
%%writefile models/research/object_detection/generate_tfrecord.py

"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record
"""

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('image_dir', '', 'Path to the image directory')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'start_tick':
        return 1
    elif row_label == 'start_gate':
        return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

In [0]:
!python models/research/object_detection/generate_tfrecord.py --csv_input=models/research/object_detection/images/train_labels.csv --image_dir=models/research/object_detection/images/train --output_path=models/research/object_detection/train.record
!python models/research/object_detection/generate_tfrecord.py --csv_input=models/research/object_detection/images/test_labels.csv --image_dir=models/research/object_detection/images/test --output_path=models/research/object_detection/test.record

Create label map

In [0]:
%%writefile models/research/object_detection/training/labelmap.pbtxt

item {
  id: 1
  name: 'start_tick'
}

item {
  id: 2
  name: 'start_gate'
}

Configure training pipeline (edit config file for pre-trained model)

In [0]:
!cp models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config models/research/object_detection/training

In [0]:
!cp models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config models/research/object_detection

In [0]:
!cat models/research/object_detection/training/faster_rcnn_inception_v2_pets.config

In [0]:
%%writefile models/research/object_detection/training/faster_rcnn_inception_v2_pets.config

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 2
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}


train_input_reader: {
  tf_record_input_reader {
    input_path: "models/research/object_detection/train.record"
  }
  label_map_path: "models/research/object_detection/training/labelmap.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 74
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "models/research/object_detection/test.record"
  }
  label_map_path: "models/research/object_detection/training/labelmap.pbtxt"
  shuffle: false
  num_readers: 1
}

In [0]:
%%writefile models/research/object_detection/faster_rcnn_inception_v2_pets.config

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 2
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}


train_input_reader: {
  tf_record_input_reader {
    input_path: "models/research/object_detection/train.record"
  }
  label_map_path: "models/research/object_detection/training/labelmap.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 74
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "models/research/object_detection/test.record"
  }
  label_map_path: "models/research/object_detection/training/labelmap.pbtxt"
  shuffle: false
  num_readers: 1
}

Running training

In [0]:
!set PYTHONPATH=models;models/research;models/research/slim

In [0]:
!models/research/setup.py build
!models/research/setup.py install

In [0]:
%%writefile -a .bashrc
export PYTHONPATH="$PYTHONPATH:models/research:models/research/slim"

In [0]:
#! /bin/bash   
!set -m
!set PYTHONPATH=models;models/research;models/research/slim
!set PATH=%PATH%;%PYTHONPATH%

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [0]:
!python models/research/object_detection/legacy/train.py --logtostderr --train_dir=models/research/object_detection/training/ --pipeline_config_path=models/research/object_detection/faster_rcnn_inception_v2_pets.config

In [0]:
%pycat /usr/local/lib/python3.6/dist-packages/object_detection/eval_util.py

In [0]:
!pip install tf_slim

Saving as frozen inference graph


In [0]:
!ls models/research/object_detection/training

In [0]:
!python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path models/research/object_detection/training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix models/research/object_detection/training/model.ckpt-36868 --output_directory models/research/object_detection/inference_graph

Mounting all info to Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp -r /content/models/* '/content/gdrive/My Drive/Gate_Detection/models'

In [0]:
!ls models/research/object_detection/inference_graph/

Testing image and video data

In [0]:
%pycat models/research/object_detection/Object_detection_image.py

In [0]:
!pip install utils

In [0]:
%%writefile models/research/object_detection/Object_detection_image_frame89.py
######## Image Object Detection Using Tensorflow-trained Classifier #########
#
# Author: Evan Juras
# Date: 1/15/18
# Description: 
# This program uses a TensorFlow-trained neural network to perform object detection.
# It loads the classifier and uses it to perform object detection on an image.
# It draws boxes, scores, and labels around the objects of interest in the image.

## Some of the code is copied from Google's example at
## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

## and some is copied from Dat Tran's example at
## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py

## but I changed it to make it more understandable to me.

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'images/test/frame89.jpg'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 2

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visualize the results')

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.60)

cv2.imwrite('/content/gdrive/My Drive/Gate_Detection/frame89_labeled.jpg', image)

# All the results have been drawn on image. Now display the image.
#cv2.imshow('Object detector', image)

# Press any key to close the image
#cv2.waitKey(0)

# Clean up
#cv2.destroyAllWindows()

In [0]:
%%bash
cd models/research/object_detection
python Object_detection_image_frame89.py

In [0]:
!rm -rf models

In [0]:
%%bash
mkdir models
cd models
mkdir research
cd research
mkdir object_detection

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp -r /content/gdrive/My\ Drive/Gate_Detection/models/research/object_detection/* '/content/models/research/object_detection'

In [0]:
%pycat models/research/object_detection/Object_detection_video.py

In [0]:
!cp '/content/gdrive/My Drive/CS/CV Footage (Duke 2019)/alpha_073019_0900/approach2.mp4' 'models/research/object_detection'

In [0]:
%%writefile models/research/object_detection/Object_detection_video_approach2.py
######## Video Object Detection Using Tensorflow-trained Classifier #########
#
# Author: Evan Juras
# Date: 1/16/18
# Description: 
# This program uses a TensorFlow-trained classifier to perform object detection.
# It loads the classifier and uses it to perform object detection on a video.
# It draws boxes, scores, and labels around the objects of interest in each
# frame of the video.

## Some of the code is copied from Google's example at
## https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

## and some is copied from Dat Tran's example at
## https://github.com/datitran/object_detector_app/blob/master/object_detection_app.py

## but I changed it to make it more understandable to me.

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

#from cv2.cv import *

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
VIDEO_NAME = 'approach2.mp4'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to video
PATH_TO_VIDEO = os.path.join(CWD_PATH,VIDEO_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 2

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Open video file
video = cv2.VideoCapture(PATH_TO_VIDEO)

ret, frame = video.read()
height, width, layers = frame.shape

fps = video.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

#label_video = cv2.VideoWriter('content/gdrive/My Drive/Gate_Detection/approach2.avi', 0, 1, (width,height))
label_video = cv2.VideoWriter('approach2.avi', fourcc, fps, (width,height))

while(video.isOpened()):

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value

    print("Reading image")

    ret, frame = video.read()
    frame_expanded = np.expand_dims(frame, axis=0)

    print("Performing object detection")

    # Perform the actual detection by running the model with the image as input
    #if frame_expanded is not None:
    if ret == True:
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})
    else:
        break
    
    print("Draw results")

    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.60)

    print("Write to video")

    # All the results have been drawn on the frame, so it's time to display it.
    #cv2.imwrite('/content/gdrive/My Drive/Gate_Detection/approach1.jpg', frame)
    #cv2.imshow('Object detector', frame)
    label_video.write(frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        print("Quit")
        break

# Clean up
label_video.release()
video.release()
cv2.destroyAllWindows()

In [0]:
%%bash
cd models/research/object_detection
python Object_detection_video_approach2.py

In [0]:
%%bash
cd models/research/object_detection/
ls

In [0]:
from google.colab import files
files.download('models/research/object_detection/approach2.avi')

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
!unzip -uq "/content/drive/My Drive/approach1_frames.zip" -d "models/research/object_detection/test_images"

In [0]:
!unzip -uq "/content/drive/My Drive/approach1_sample.zip" -d "models/research/object_detection/test_images"

In [0]:
!unzip -uq "/content/drive/My Drive/approach1_sample_saturated.zip" -d "models/research/object_detection/test_images"

In [0]:
!unzip -uq "/content/drive/My Drive/approach1_sample_super_saturated.zip" -d "models/research/object_detection/test_images"

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/approach1_sample')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/approach1_sample_saturated')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/appraoch1_sample_super_saturated')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/appraoch1_sample_super_saturated')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/My Drive/Normal_Photo')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
!cd models/research/object_detection/test_images

In [0]:
!ls -A

# Detection

Load an object detection model:

In [0]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [0]:
print(detection_model.inputs)

And retuns several outputs:

In [0]:
detection_model.output_dtypes

In [0]:
detection_model.output_shapes

Add a wrapper function to call the model, and cleanup the outputs:

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


## Instance Segmentation

In [0]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model("mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28")

The instance segmentation model includes a `detection_masks` output:

In [0]:
masking_model.output_shapes

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)

Training the autonomous submarine model

In [0]:
#Getting xml files into model
